In [1]:
import datetime
import itertools
from itertools import product
import math
import matplotlib.pyplot as plt
from multiprocessing.dummy import Pool as ThreadPool
import nolds
import numpy as np
import os
import pandas as pd
import pathlib
from PreprocessFcns import *
import pywt
import random
from scipy.signal import butter, welch, filtfilt, resample
from scipy.stats import skew, kurtosis, entropy, pearsonr
import seaborn as sns
import sklearn
from sklearn import multiclass
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneGroupOut
import time
%matplotlib inline

In [2]:
# set path to folder containing clinic watch data
clinicpath = r'//FS2.smpp.local\\RTO\\CIS-PD Study\MJFF Curation\Finalized Dataset'
# set path to folder containing home watch data
homepath = r'//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Record Correlation'
# set path to destination folder
dest = r'//FS2.smpp.local\\RTO\\CIS-PD Study\Watch Features Data'
#---------------------------------------------------------------------------------------------------------

In [3]:
table_diar = 'Table12.csv'
dest_ext_diar = 'Diaries'
file_name_diar = 'diar_timepoints.csv'

timepoints_diar = pd.read_csv(os.path.join(homepath, dest_ext_diar, file_name_diar), parse_dates = [1])

In [4]:
def getDiaryTimePath(timepoints_diar):
    
    # Initialize Empty Lists for Each Necessary Piece of Information from Each Record
    StartTimestamps = []
    EndTimestamps = []
    SaveFilePaths = []
    SubjID = []
    
    # Diaries
    Subj = list(timepoints_diar.SubjID)
    StartTimes = timepoints_diar.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = -30), axis = 1)
    EndTimes = timepoints_diar.apply(lambda row: row.Timestamp, axis = 1)
    SavePaths = timepoints_diar.apply(lambda row: os.path.join(homepath, dest_ext_diar, str(row.SubjID), 
                                                                                        str(row.Timestamp)[:7], 
                                                                                        str(row.Timestamp)[8:10],
                                                                                        str(row.Timestamp)[11:13] +
                                                                                        str(row.Timestamp)[14:16] +
                                                                                        str(row.Timestamp)[17:] + '.csv'), 
                                      axis = 1)
    SubjID = SubjID + Subj
    StartTimestamps = StartTimestamps + list(StartTimes)
    EndTimestamps = EndTimestamps + list(EndTimes)
    SaveFilePaths = SaveFilePaths + list(SavePaths)
    
    DiaryTimePath = pd.DataFrame(columns = ['SubjID', 'StartTimestamps', 'EndTimestamps', 'SaveFilePaths'])
    
    DiaryTimePath['SubjID'] = SubjID
    DiaryTimePath['StartTimestamps'] = StartTimestamps
    DiaryTimePath['EndTimestamps'] = EndTimestamps
    DiaryTimePath['SaveFilePaths'] = SaveFilePaths
    
    return DiaryTimePath

In [5]:
DiaryTimePath = getDiaryTimePath(timepoints_diar)
print(DiaryTimePath.SaveFilePaths[100])
DiaryTimePath.head()

//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Record Correlation\Diaries\1046\2017-10\03\170000.csv


,SubjID,StartTimestamps,EndTimestamps,SaveFilePaths
0,1046,2017-09-18 12:30:00,2017-09-18 13:00:00,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...
1,1046,2017-09-18 13:00:00,2017-09-18 13:30:00,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...
2,1046,2017-09-18 13:30:00,2017-09-18 14:00:00,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...
3,1046,2017-09-18 14:00:00,2017-09-18 14:30:00,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...
4,1046,2017-09-18 14:30:00,2017-09-18 15:00:00,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...


In [6]:
DiaryRecords = (pd.read_csv(os.path.join(clinicpath, 'Metadata Tables', 'Table12.csv'), parse_dates = ['Timestamp']) 
                [['SubjID', 'Timestamp', 'Measurement Name', 'Value']])
print(DiaryRecords.head())

   SubjID           Timestamp    Measurement Name  Value
0    1046 2017-09-18 13:00:00              On/Off   -1.0
1    1046 2017-09-18 13:30:00              Tremor    1.0
2    1046 2017-09-18 13:30:00              On/Off    0.0
3    1046 2017-09-18 13:30:00          Dyskinesia    0.0
4    1046 2017-09-18 13:30:00  Activity Intensity    1.0


In [8]:
NoSympDiar = pd.DataFrame()
for sub in DiaryRecords.SubjID.unique():
    subDiaryRecords = DiaryRecords[DiaryRecords.SubjID == sub]
    for time in subDiaryRecords.Timestamp.unique():
        timesubDiaryRecords = subDiaryRecords[subDiaryRecords.Timestamp == time]
        ind = ((timesubDiaryRecords['Measurement Name'] == 'Tremor') | 
               (timesubDiaryRecords['Measurement Name'] == 'On/Off') | 
               (timesubDiaryRecords['Measurement Name'] == 'Dyskinesia'))
        if all(value == float(0) for value in timesubDiaryRecords.Value[ind]):
            time = pd.Timestamp(time)
            NoSymp = pd.DataFrame()
            NoSymp['Path'] = [os.path.join(homepath, dest_ext_diar, 
                                           str(sub), str(time)[:7], str(time)[8:10],
                                           str(time)[11:13] +
                                           str(time)[14:16] +
                                           str(time)[17:] + '.csv')]
            NoSymp['SubjID'] = [sub]
            NoSymp['On/Off'] = [np.max(timesubDiaryRecords.Value[timesubDiaryRecords['Measurement Name'] == 
                                                                 'On/Off'].values)]
            NoSymp['Tremor'] = [np.max(timesubDiaryRecords.Value[timesubDiaryRecords['Measurement Name'] == 
                                                                 'Tremor'].values)]
            NoSymp['Dyskinesia'] = [np.max(timesubDiaryRecords.Value[timesubDiaryRecords['Measurement Name'] == 
                                                                     'Dyskinesia'].values)]
            NoSymp['Activity Intensity'] = [np.max(timesubDiaryRecords.Value[timesubDiaryRecords['Measurement Name'] == 
                                                                             'Activity Intensity'].values)]
            if NoSympDiar.empty:
                NoSympDiar = NoSymp
                continue
            NoSympDiar = pd.concat([NoSympDiar, NoSymp], ignore_index = True)
NoSympDiar.head()

,Path,SubjID,On/Off,Tremor,Dyskinesia,Activity Intensity
0,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,1046,0.0,0.0,0.0,2.0
1,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,1046,0.0,0.0,0.0,1.0
2,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,1046,0.0,0.0,0.0,1.0
3,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,1046,0.0,0.0,0.0,1.0
4,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,1046,0.0,0.0,0.0,1.0


In [9]:
NoSympDiar.Path[1]

'//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Diaries\\1046\\2017-09\\18\\203000.csv'

In [10]:
def feature_extraction(data_clip):
    
    features_list = ['RMSX', 'RMSY', 'RMSZ', 
                     'rangeX', 'rangeY', 'rangeZ', 
                     'meanX', 'meanY', 'meanZ', 
                     'varX', 'varY', 'varZ', 
                     'skewX', 'skewY', 'skewZ', 
                     'kurtX', 'kurtY', 'kurtZ', 
                     'xcorr_peakXY', 'xcorr_peakXZ', 'xcorr_peakYZ', 
                     'xcorr_lagXY', 'xcorr_lagXZ', 'xcorr_lagYZ', 
                     'freq1', 'freq2', 'freq3', # added
                     'P1_rel', 'P2_rel', 'P3_rel', # added
                     'PSD_mean', 'PSD_std', 'PSD_skew', 'PSD_kur', 
                     'jerk_mean', 'jerk_std', 'jerk_skew', 'jerk_kur', 
                     'Sen_X', 'Sen_Y', 'Sen_Z', # not included in reduced features
                     'RMS_mag', 'range_mag', 'mean_mag', 'var_mag', 'skew_mag', 'kurt_mag', 'Sen_mag'] # not included
    
    rawdata = data_clip
    rawdata_wmag = rawdata.copy()
    rawdata_wmag['Accel_Mag'] = np.sort((rawdata**2).sum(axis = 1))
    
    N = len(rawdata)
    min_xyz = np.min(rawdata, axis = 0)
    max_xyz = np.max(rawdata, axis = 0)
    xcorr_xy = np.correlate(rawdata.iloc[:, 0], rawdata.iloc[:, 1], mode = 'same')
    xcorr_xz = np.correlate(rawdata.iloc[:,0], rawdata.iloc[:, 2], mode = 'same')
    xcorr_yz = np.correlate(rawdata.iloc[:, 1], rawdata.iloc[:, 2], mode = 'same')
    Pxx = power_spectra_welch(rawdata_wmag, fm = 0, fM = 10)
    sH_raw = []; sH_fft = []

    
    # RMSX, RMSY, RMSZ
    # root mean square
    RMS = 1 / N * np.sqrt(np.asarray(np.sum(rawdata**2, axis = 0)))
    
    # rangeX, rangeY, rangeZ
    # range
    r = np.asarray(max_xyz - min_xyz)
    
    # meanX, meanY, meanZ
    # average
    mean = np.asarray(np.mean(rawdata, axis = 0))

    # varX, varY, varZ
    # standard deviation
    var = np.asarray(np.std(rawdata, axis = 0))

    # skewX, skewY, skewZ
    # skewness: measure of data symmetry
    sk = skew(rawdata)
    
    # kurtX, kurtY, kurtZ
    # kurtosis: measure of data tail weight compared to normal dist
    kurt = kurtosis(rawdata)

    # xcorr_peakXY, xcorr_peakXZ, xcorr_peakYZ
    # max correlation between data of paired axes
    xcorr_peak_xy = np.max(xcorr_xy)
    xcorr_peak_xz = np.max(xcorr_xz)
    xcorr_peak_yz = np.max(xcorr_yz)
    xcorr_peak = np.array([xcorr_peak_xy, xcorr_peak_xz, xcorr_peak_yz])

    # xcorr_lagXY, xcorr_lagXZ, xcorr_lagYZ
    # relative location of max correlation between data of paired axes
    xcorr_lag_xy = (np.argmax(xcorr_xy)) / len(xcorr_xy)
    xcorr_lag_xz = (np.argmax(xcorr_xz)) / len(xcorr_xz)
    xcorr_lag_yz = (np.argmax(xcorr_yz)) / len(xcorr_yz)
    xcorr_lag = np.array([xcorr_lag_xy, xcorr_lag_xz, xcorr_lag_yz])

    # freq1, freq2, freq3
    # frequency with the highest power density
    freq1 = Pxx.iloc[:, -1].index[-1]
    freq2 = Pxx.iloc[:, -1].index[-2]
    freq3 = Pxx.iloc[:, -1].index[-3]
    domfreq = np.array([freq1, freq2, freq3])

    # P1_rel, P2_rel, P3_rel
    # relative power of the dominant frequency within the signal
    P1_rel = Pxx.loc[freq1].values / Pxx.iloc[:, -1].sum()
    P2_rel = Pxx.loc[freq2].values / Pxx.iloc[:, -1].sum()
    P3_rel = Pxx.loc[freq3].values / Pxx.iloc[:, -1].sum()
    Pdom_rel = np.concatenate((P1_rel, P2_rel, P3_rel))

    # PDS_mean, PDS_std, PDS_skew, PDS_kur
    # power spectral density summary stats
    Pxx_moments = np.array([np.nanmean(Pxx.values), np.nanstd(Pxx.values), skew(Pxx.values), kurtosis(Pxx.values)])

    # jerk_mean, jerk_std, jerk_skew, jerk_kur
    jerk = rawdata_wmag['Accel_Mag'].diff().values
    jerk_moments = np.array([np.nanmean(jerk), np.nanstd(jerk), skew(jerk[~np.isnan(jerk)]), kurtosis(jerk[~np.isnan(jerk)])])

    # Sen_X, Sen_Y, Sen_Z
    # sample entropy
    for a in range(3):
        x = rawdata.iloc[:, a]
        n = len(x)
        Fs = np.mean(1 / (np.diff(x.index) / 1000))
        sH_raw.append(nolds.sampen(x))
    
    # features of the acceleration magnitude (as opposed to the axis values)
    RMS_mag = 1 / N * np.sqrt(np.sum(rawdata_wmag['Accel_Mag']**2, axis = 0))
    r_mag = np.max(rawdata_wmag['Accel_Mag']) - np.min(rawdata_wmag['Accel_Mag'])
    mean_mag = np.mean(rawdata_wmag['Accel_Mag'])
    var_mag = np.std(rawdata_wmag['Accel_Mag'])
    sk_mag = skew(rawdata_wmag['Accel_Mag'])
    kurt_mag = kurtosis(rawdata_wmag['Accel_Mag'])
    sH_mag = nolds.sampen(rawdata_wmag['Accel_Mag'])
    
    Y = np.array([RMS_mag, r_mag, mean_mag, var_mag, sk_mag, kurt_mag, sH_mag])
    X = np.concatenate((RMS, r, mean, var, sk, kurt, xcorr_peak, xcorr_lag, 
                        domfreq, Pdom_rel, Pxx_moments, jerk_moments, sH_raw, Y))
    
    return X

In [11]:
def DataAggregator(file, data_type):
    '''generates feature metrics for 5-second intervals of apple watch data
       utilizes additional features not present in previous DataAggregator
       data_type: \'clinic\' or \'home\'
       file: clinic data in the form (SubjID)_(VisitNum)_(TaskAbb); home data as full record file path'''
    
    TaskFeatures = pd.DataFrame()
    
    if data_type == 'clinic':
        subject = file[:4]
        visit = file[5:6]
        task = file[7:]
    if data_type == 'home':
        designation = file[63:-4]
    print(file)
    
    features_list = ['RMSX', 'RMSY', 'RMSZ', 
                     'rangeX', 'rangeY', 'rangeZ', 
                     'meanX', 'meanY', 'meanZ', 
                     'varX', 'varY', 'varZ', 
                     'skewX', 'skewY', 'skewZ', 
                     'kurtX', 'kurtY', 'kurtZ', 
                     'xcorr_peakXY', 'xcorr_peakXZ', 'xcorr_peakYZ', 
                     'xcorr_lagXY', 'xcorr_lagXZ', 'xcorr_lagYZ', 
                     'freq1', 'freq2', 'freq3', # added
                     'P1_rel', 'P2_rel', 'P3_rel', # added
                     'PSD_mean', 'PSD_std', 'PSD_skew', 'PSD_kur', 
                     'jerk_mean', 'jerk_std', 'jerk_skew', 'jerk_kur', 
                     'Sen_X', 'Sen_Y', 'Sen_Z', # not included in reduced features
                     'RMS_mag', 'range_mag', 'mean_mag', 'var_mag', 'skew_mag', 'kurt_mag', 'Sen_mag'] # not included
        
    # get acc data
    try:
        if data_type == 'clinic':
            data = (pd.read_csv(os.path.join(clinicpath, 'Table8', 'TaskAcc', file + '.csv'), parse_dates = ['timestamp'])
                    [['timestamp', 'x', 'y', 'z']])
            data.columns = ['Timestamp', 'X', 'Y', 'Z']
        if data_type == 'home':
            data = pd.read_csv(file, parse_dates = ['Timestamp'])[['Timestamp', 'X', 'Y', 'Z']]
    except(FileNotFoundError):
        if data_type == 'clinic':
            print('No data found for subject %s %s visit %s' % (subject, task, visit))
        if data_type == 'home':
            print('No data found for %s' % (designation))
        return TaskFeatures
        
    # organize data and make 5 second clips
    data = data.sort_values(by = 'Timestamp', axis = 0)
    data['Timestamp2'] = [(tm - datetime.timedelta(minutes = 0,
                                                   seconds = tm.second % 5,
                                                   microseconds = tm.microsecond)) 
                          for tm in data.Timestamp]
    
    data['Timestamp'] = (data.Timestamp.values - data.Timestamp.values[0]).astype('timedelta64[ms]').astype(int)
    data = data.set_index('Timestamp')
    data.loc[:, ['X', 'Y', 'Z']] = filterdata(data[['X', 'Y', 'Z']])
                
    # "clip" the data into 5 second chunks    
    five_sec_intervals = data.Timestamp2.unique()
        
    # calculate features
    F = []
    num_empty = 0
    times = []
    for t in five_sec_intervals:
        clip = data.loc[(data.Timestamp2 == t)]
        # length of 5 second chunk should be 250 for 5 seconds
        if (clip.empty or (len(clip.Timestamp2) < 200)):
            num_empty += 1
        else:
            F.append(feature_extraction(clip[['X', 'Y', 'Z']]))
            times.append(t)
            
    # create features dataframe
    TaskFeatures = pd.DataFrame(data = F, columns = features_list, dtype = 'float32')    
    
    return TaskFeatures

In [ ]:
### GENERATING FEATURES FROM HOME DATA RELATED TO DIARIES
s_time = time.time()

DataFeatures = pd.DataFrame()

for diary in NoSympDiar.iterrows():
    file = diary[1]['Path']
    TaskFeatures = DataAggregator(file, 'home')
    if TaskFeatures.empty:
        continue
    featcols = list(TaskFeatures.columns)
    
    TaskFeatures['SubjID'] = diary[1]['SubjID']
    TaskFeatures['On/Off'] = diary[1]['On/Off']
    TaskFeatures['Tremor'] = diary[1]['Tremor']
    TaskFeatures['Dyskinesia'] = diary[1]['Dyskinesia']
    TaskFeatures['Activity Intensity'] = diary[1]['Activity Intensity']
    
    cols = ['SubjID', 'On/Off', 'Tremor', 'Dyskinesia', 'Activity Intensity'] + featcols
    TaskFeatures = TaskFeatures[cols]
    
    if DataFeatures.empty:
        DataFeatures = TaskFeatures
        continue
    DataFeatures = pd.concat([DataFeatures, TaskFeatures], ignore_index = True)
    
DataFeatures.to_csv(os.path.join(dest, 'Home Data', 'DiaryDataFeatures_NoSymp.csv'), index = False)
    
print(str(int(((time.time() - s_time) / 60) / 60)) + ' hours ' + 
      str(int(((time.time() - s_time) / 60) % 60)) + ' minutes ' + 
      str(int((time.time() - s_time) % 60)) + ' seconds')

//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Record Correlation\Diaries\1046\2017-09\18\140000.csv


In [79]:
DiaryRecords['Measurement Name'].unique()

array(['On/Off', 'Tremor', 'Dyskinesia', 'Activity Intensity'],
      dtype=object)